In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time


# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [2]:
x_temp = np.array([0, 1, 2, 3, 4])
x_temp[-2:]

array([3, 4])

In [3]:
arr = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]])
# num_cols = arr.shape[1]
num_cols = 3
# arr[:, -num_cols:] - arr[:, -num_cols-1:-1]

np.linalg.norm(arr, axis=0)


array([12.56980509, 14.03566885, 15.55634919, 17.11724277, 18.70828693])

In [4]:
grad = np.array([0, 1, 2, 3, 4])

grad[2:]

array([2, 3, 4])

In [5]:

def solve_CBCP_direct_11(tau, B, v_I, v_E, a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

    # y_el: \hat y_1, \tilde y_1, y_2
    y_el = cp.Variable(3)
    # y_in: y_1, y_2
    y_in = cp.Variable(2)
    
    func = 1/5 * a[4] * cp.power(y_el[0] + y_el[1] + y_in[0], 5)
    func += 1/4 * a[3] * cp.power(y_el[0] + y_el[1] + y_in[0], 4)
    func += 1/3 * a[2] * cp.power(y_el[0] + y_el[1] + y_in[0], 3)
    func += 1/2 * a[1] * cp.power(y_el[0] + y_el[1] + y_in[0], 2)
    func += a[0] * (y_el[0] + y_el[1] + y_in[0])
    func += y_in[0] * tau / v_I + y_el[0] * tau / v_E
    func += 1/5 * a[4] * cp.power(y_el[2] + y_in[1], 5)
    func += 1/4 * a[3] * cp.power(y_el[2] + y_in[1], 4)
    func += 1/3 * a[2] * cp.power(y_el[2] + y_in[1], 3)
    func += 1/2 * a[1] * cp.power(y_el[2] + y_in[1], 2)
    func += a[0] * (y_el[2] + y_in[1])

    objective = cp.Minimize(func)
    
    constraints = []
    constraints += [y_el >= 0, y_in >= 0]
    constraints += [cp.sum(y_el) == 1, cp.sum(y_in) == 1]
    constraints += [y_el[1] * tau <= B]
    
    prob = cp.Problem(objective, constraints)
    
    # The optimal objective value is returned by `prob.solve()`.
    result = prob.solve()
#     # The optimal value for x is stored in `x.value`.
#     print("In solve_CBCP_direct, y_el.value:", y_el.value)
#     print("In solve_CBCP_direct, y_in.value:", y_in.value)

    return y_el.value, y_in.value


In [6]:
def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I, v_E, y_el, y_in, a):
    
    # y = (\hat y_1 E, \tilde y_1 E, y_2 E, y_1 I, y_2 I)
    # y_el = (\hat y_1 E, \tilde y_1 E, y_2 E)
    # y_in = (y_1 I, y_2 I)
    
    ell_1 = a[4] * (y_el[0] + y_el[1] + y_in[0])**4 \
            + a[3] * (y_el[0] + y_el[1] + y_in[0])**3 \
            + a[2] * (y_el[0] + y_el[1] + y_in[0])**2 \
            + a[1] * (y_el[0] + y_el[1] + y_in[0]) \
            + a[0]
        
    ell_2 = a[4] * (y_el[2] + y_in[1])**4 \
            + a[3] * (y_el[2] + y_in[1])**3 \
            + a[2] * (y_el[2] + y_in[1])**2 \
            + a[1] * (y_el[2] + y_in[1]) \
            + a[0]
    
    obj_E = tau * y_el[0] + v_E * (ell_1 * (y_el[0] + y_el[1]) + ell_2 * y_el[2])
    obj_R = tau * (y_el[0] + y_in[0])
    obj_I = tau * y_in[0] + v_I * (ell_1 * y_in[0] + ell_2 * y_in[1] )
    
#     print("ell_1:", ell_1)
#     print("ell_2:", ell_2)
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)

#     print("tau:", tau)
#     print("y_in[0]:", y_in[0])
#     print("v_I:", v_I)
#     print("ell_1:", ell_1)
#     print("y_el[0] + y_el[1] + y_el[2]:", y_el[0] + y_el[1] + y_el[2])
#     print("ell_2:", ell_2)
#     print("y_in[0] + y_in[1]:", y_in[0] + y_in[1])

#     print("obj_I:", obj_I)

    return lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    

In [7]:
def proj_tau_B_11(tau, B):
    
    # TODO:
    tau_feas = cp.Variable(1)
    B_feas = cp.Variable(1)
    
    func = (tau_feas - tau)**2 + (B_feas - B)**2

    objective = cp.Minimize(func)
    
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0]
    constraints += [B_feas >= 0]
    constraints += [tau_feas <= 1]
    constraints += [B_feas <= 1]
    constraints += [B_feas <= tau_feas]
    
    prob = cp.Problem(objective, constraints)
    
    result = prob.solve()

    return tau_feas.value, B_feas.value

## Original Code below:

# def proj_tau_B_11(tau, B):
#     if B <= tau and 0 <= tau <= 1 and 0 <= B <= 1:
#         return tau, B
#     # Case 1:
#     elif tau > 1 and 0 < B <= 1:
#         return 1.0, B
#     # Case 2:
#     elif tau > 1 and B <= 0:
#         return 1.0, 0.0
#     # Case 3:
#     elif 0 < tau <= 1 and B < 0:
#         return tau, 0.0
#     # Case 4:
#     elif tau <= 0 and B + tau < 0:
#         return 0.0, 0.0
#     # Case 5:
#     elif 0 <= B + tau < 2 and B > tau:
#         return (B + tau)/2, (B + tau)/2
#     # Case 6:
#     elif B + tau >= 2 and B > 1:
#         return 1.0, 1.0
#     else:
#         assert 1 == 0, "This case should not happen!"

# Chinmay's Algorithm:

In [8]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I = 1.0
v_E = 0.6
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

tau_max, B_max = 1.0, 1.0
d = 2
num_iters = 1000
tau = np.zeros(num_iters)
tau_perturbed = np.zeros(num_iters)
B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
tau_B_iters = np.zeros((2, num_iters))
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 0.5
delta_bar = 0.5

welfare_list = []

tau[0] = 0.8
B[0] = 0.2

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(2)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[i] = tau[i] + delta[i] * w_i[0]
    B_perturbed[i] = B[i] + delta[i] * w_i[1]
    
#     if tau_perturbed[i] < B_perturbed[i] or tau_perturbed[i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[i], B_perturbed[i] = proj_tau_B_11(tau_perturbed[i], B_perturbed[i])
    
    # TODO
#     if B_perturbed[i] <= tau_perturbed[i] and B_perturbed[i] >= 0 and tau_perturbed[i] >= 0:        
    
#     print()
#     print("New solve_CBCP_FW_11 call to solve_CBCP (unperturbed):")
#     print()
    print("tau[i]:", tau[i])
    print("B[i]:", B[i])
    print("tau_perturbed[i]:", tau_perturbed[i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el, y_in = solve_CBCP_direct_11(tau = tau[i], B = B[i], v_I = v_I, v_E = v_E, a = a)
    

#     print()
#     print("New solve_CBCP_FW_11 call to solve_CBCP (perturbed):")
#     print()
    y_el_perturbed, y_in_perturbed = solve_CBCP_direct_11(tau = tau_perturbed[i], B = B_perturbed[i], v_I = v_I, v_E = v_E, a = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau[i], v_I = v_I, v_E = v_E, \
                          y_el = y_el, y_in = y_in, a = a)
    welfare_perturbed = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau_perturbed[i], v_I = v_I, v_E = v_E, \
                                    y_el = y_el_perturbed, y_in = y_in_perturbed, a = a)
    
    welfare_list.append(welfare)
    
    print("welfare:", welfare)
    print("welfare_perturbed:", welfare_perturbed)
#     print("tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare):\n", tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare))
    
    tau[i+1] = tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare)
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[1] * (welfare_perturbed - welfare)

    if not (B[i+1] <= tau[i+1] and 0 <= tau[i+1] <= 1 and 0 <= B[i+1] <= 1):
        tau[i+1], B[i+1] = proj_tau_B_11(tau[i+1], B[i+1])
        
    if i >= diffs_num_cols + 2:
        tau_diffs = tau[i-diffs_num_cols : i-1] - tau[i-diffs_num_cols+1 : i]
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[0:10]:", tau[0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
w_i: [ 0.6296369  -0.77688955]
tau[i]: 0.8
B[i]: 0.2
tau_perturbed[i]: 1.0
B_perturbed[i]: -1.9626078274491337e-23
welfare: 1.6935225534581637
welfare_perturbed: 1.6690338122931614

Iter: 1
w_i: [ 0.21281785 -0.97709189]
tau[i]: 0.8218057800819524
B[i]: 0.1730945202103644
tau_perturbed[i]: 0.8850769049715571
B_perturbed[i]: -1.643588614540067e-30
welfare: 1.6878479220933902
welfare_perturbed: 1.6347032148599614

Iter: 2
w_i: [ 0.86032293 -0.5097494 ]
tau[i]: 0.8352558817163034
B[i]: 0.11134225020006919
tau_perturbed[i]: 1.0
B_perturbed[i]: -3.583674144018549e-31
welfare: 1.6664040522143664
welfare_perturbed: 1.6690338122931614

Iter: 3
w_i: [0.15518935 0.98788474]
tau[i]: 0.832824728600448
B[i]: 0.11278273120732643
tau_perturbed[i]: 0.8716220653055842
B_perturbed[i]: 0.35975391710679183
welfare: 1.666384714800675
welfare_perturbed: 1.7750390038024053

Iter: 4
w_i: [ 0.06554392 -0.99784969]
tau[i]: 0.8159627404610187
B[i]: 0.005444816776044123
tau_perturbed[i]: 0.83145964401379

welfare: 1.5754912674607673
welfare_perturbed: 1.616448826593192
tau_diffs: [ 1.81877795e-02  8.36817471e-05 -3.00033265e-04  4.14973925e-03]
B_diffs: [-1.01901373e-03 -8.99718981e-04  1.91873271e-03  9.81303812e-24]

Iter: 26
w_i: [ 0.67844757 -0.73464882]
tau[i]: 0.5526560986540191
B[i]: -1.0437336592363576e-31
tau_perturbed[i]: 0.6578838664353905
B_perturbed[i]: -1.5952477729359475e-30
welfare: 1.5728830473055302
welfare_perturbed: 1.5862917544804873
tau_diffs: [ 8.36817471e-05 -3.00033265e-04  4.14973925e-03  7.38586236e-05]
B_diffs: [-8.99718981e-04  1.91873271e-03  9.81303812e-24 -3.06657505e-25]

Iter: 27
w_i: [-0.03562035  0.99936539]
tau[i]: 0.5470122253545999
B[i]: 0.006111400569666414
tau_perturbed[i]: 0.5415374837779326
B_perturbed[i]: 0.15971090176594444
welfare: 1.5747757295372167
welfare_perturbed: 1.635695830383225
tau_diffs: [-3.00033265e-04  4.14973925e-03  7.38586236e-05  2.45437531e-02]
B_diffs: [ 1.91873271e-03  9.81303812e-24 -3.06657505e-25  3.06657549e-25]

Iter

welfare: 1.5688435801745255
welfare_perturbed: 1.6129580984930356
tau_diffs: [0.00400644 0.00286202 0.00358699 0.01507846]
B_diffs: [-6.13314942e-25 -9.54031457e-05  9.54031457e-05  4.35726604e-32]

Iter: 50
w_i: [-0.21621895  0.9763449 ]
tau[i]: 0.5169754062990789
B[i]: -2.9418579900515028e-31
tau_perturbed[i]: 0.4883694671809448
B_perturbed[i]: 0.12917120924385486
welfare: 1.5696587726978946
welfare_perturbed: 1.619233956023302
tau_diffs: [0.00286202 0.00358699 0.01507846 0.00289498]
B_diffs: [-9.54031457e-05  9.54031457e-05  4.35726604e-32 -4.76562337e-32]

Iter: 51
w_i: [ 0.06476274 -0.99790069]
tau[i]: 0.5226479850029867
B[i]: -3.586081556449033e-31
tau_perturbed[i]: 0.5311746531621611
B_perturbed[i]: -1.8392630968898102e-30
welfare: 1.570126307120523
welfare_perturbed: 1.5708611612291985
tau_diffs: [ 0.00358699  0.01507846  0.00289498 -0.01043145]
B_diffs: [ 9.54031457e-05  4.35726604e-32 -4.76562337e-32  2.87073400e-31]

Iter: 52
w_i: [0.99341589 0.11456378]
tau[i]: 0.5226229215

welfare: 1.5658978340987244
welfare_perturbed: 1.5717484436341316
tau_diffs: [0.01487494 0.00331164 0.01279489 0.01263551]
B_diffs: [ 4.90651918e-24  2.29426405e-31 -8.02631304e-32 -1.88283726e-31]

Iter: 73
w_i: [-0.93137482 -0.36406172]
tau[i]: 0.4578240278512188
B[i]: 0.0020980053190884592
tau_perturbed[i]: 0.34555199902776396
B_perturbed[i]: -5.850012206076229e-31
welfare: 1.5666428670357768
welfare_perturbed: 1.5636376182580138
tau_diffs: [0.00331164 0.01279489 0.01263551 0.0005954 ]
B_diffs: [ 2.29426405e-31 -8.02631304e-32 -1.88283726e-31 -5.78651670e-32]

Iter: 74
w_i: [-0.06989412 -0.99755442]
tau[i]: 0.4564744063416242
B[i]: 0.0015704566676345718
tau_perturbed[i]: 0.4480772870883051
B_perturbed[i]: 1.1496660453432662e-23
welfare: 1.5663658579406168
welfare_perturbed: 1.5653467249759017
tau_diffs: [0.01279489 0.01263551 0.0005954  0.00190024]
B_diffs: [-8.02631304e-32 -1.88283726e-31 -5.78651670e-32 -2.09800532e-03]

Iter: 75
w_i: [-0.43793817  0.89900509]
tau[i]: 0.4564401752

welfare: 1.5662516982903636
welfare_perturbed: 1.564800326901881
tau_diffs: [-0.00890055  0.00066579 -0.00740248  0.00151682]
B_diffs: [ 2.85265105e-03 -3.41972936e-32 -4.90651909e-24  5.01493898e-24]

Iter: 96
w_i: [0.97540089 0.22043842]
tau[i]: 0.4663129563035123
B[i]: -1.533287371589325e-25
tau_perturbed[i]: 0.5761996791424344
B_perturbed[i]: 0.02483415406032929
welfare: 1.5662417044168064
welfare_perturbed: 1.585318465967715
tau_diffs: [ 0.00066579 -0.00740248  0.00151682  0.01192843]
B_diffs: [-3.41972936e-32 -4.90651909e-24  5.01493898e-24 -8.26961193e-25]

Iter: 97
w_i: [-0.97390548  0.22695401]
tau[i]: 0.45792782506667673
B[i]: -2.6529684983934174e-32
tau_perturbed[i]: 0.34849054387054806
B_perturbed[i]: 0.025502711450002755
welfare: 1.5658081001367838
welfare_perturbed: 1.5738089034789189
tau_diffs: [-0.00740248  0.00151682  0.01192843  0.00018507]
B_diffs: [-4.90651909e-24  5.01493898e-24 -8.26961193e-25  8.71870032e-25]

Iter: 98
w_i: [ 0.54426123 -0.8389158 ]
tau[i]: 0.461

welfare: 1.5676034825022482
welfare_perturbed: 1.5705627189968732
tau_diffs: [-0.00709233  0.00177863  0.00017475  0.00424467]
B_diffs: [-1.08419700e-25 -7.66645559e-26  7.66643691e-26 -7.99859215e-04]

Iter: 118
w_i: [-0.04106238 -0.99915658]
tau[i]: 0.4811921496019442
B[i]: 0.002308333451792845
tau_perturbed[i]: 0.47679660851552574
B_perturbed[i]: -1.4650549086953703e-30
welfare: 1.5680278748105092
welfare_perturbed: 1.566836630867876
tau_diffs: [0.00177863 0.00017475 0.00424467 0.0041171 ]
B_diffs: [-7.66645559e-26  7.66643691e-26 -7.99859215e-04 -3.61537753e-04]

Iter: 119
w_i: [-0.99713149  0.07568877]
tau[i]: 0.481171204955169
B[i]: 0.0017986946127300277
tau_perturbed[i]: 0.37465587254893157
B_perturbed[i]: 0.009883901912465872
welfare: 1.5678228754741144
welfare_perturbed: 1.5675011537893124
tau_diffs: [0.00017475 0.00424467 0.0041171  0.00054484]
B_diffs: [ 7.66643691e-26 -7.99859215e-04 -3.61537753e-04 -1.14693648e-03]

Iter: 120
w_i: [ 0.23140799 -0.97285679]
tau[i]: 0.481034

In [9]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

first(welfare_list): 1.6935225534581637
min(welfare_list): 1.5658081001367838
max(welfare_list): 1.6935225534581637
argmin_tau: 0.45792782506667673
argmin_B: -2.6529684983934174e-32


## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct_11(tau = tau, B = B, v_I = v_I, v_E = v_E, a = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I, v_E, \
#                                                   y_el_point_5, y_in_point_5, \
#                                                   a = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I = 1.0
v_E = 0.6
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct_11(tau = tau, B = B, v_I = v_I, v_E = v_E, a = a)
            
#             solve_CBCP_direct_11(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I, v_E, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a = a)
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))
print("\nmin welfare:\n", welfare_obj_arr[argmin_indices[0], argmin_indices[1]])


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1, cp.sum(y_in_var) == 1]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
# for i in range(2):
#     constraints += [x[i] >= 2]
# constraints += [x[i] >=2 for i in range(2)]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


# Old Code Below:

# Budget vs Discount - Theory

## 1 Eligible group, 1 Ineligible group (Theory)

In [ ]:
def F_11(val, ell, tau, alpha):
    return ell(val) + (1 - alpha) * tau - ell(2 - val)

def y_star_11(ell, tau, alpha, lb_init = 0.0, ub_init = 2.0, num_iters = 20):
    # Searches for y^\star satisfying ell(y^\star) + (1 - \alpha) \tau = ell(2 - y^\star)
    
    assert tau < ell(ub_init) - ell(lb_init), "We must have tau < ell(2.0) - ell(0.0), else is trivial."
    assert 0.0 <= alpha <= 1.0, "We must have 0.0 <= alpha <= 1.0"
    assert ell(lb_init) + (1.0 - alpha) * tau < ell(ub_init)
    
    lb = lb_init
    ub = ub_init
    mid_val = (lb + ub)/2
    
    for k in range(num_iters):
#         print("F:", F_11(mid_val, ell, tau, alpha))
#         print("lb:", lb)
#         print("ub:", ub)
#         print("mid_val:", mid_val)
#         print()
        
        if F_11(mid_val, ell, tau, alpha) < 0:
            lb = mid_val
        else:
            ub = mid_val
        mid_val = (lb + ub)/2
    
    return (lb + ub)/2

# def fixed_point_eqn():
    


In [ ]:
t_1 = time.time()

ell = lambda x: x**4 / 16
# ell = lambda x: np.log(1 + np.exp(x))
# alpha = 0.0

# # Case 1: 
# tau = 0.4
# v_E = 1
# v_I = 1.25

# # Case 2: 
# tau = 0.7
# v_E = 1
# v_I = 1.25

# Case 3: 
tau = 0.7
v_E = 1
v_I = 2.5

num_alpha_grid_points = 100 # Should be even
alpha_vals = np.linspace(0, 1.0, num = num_alpha_grid_points + 1)

y_star_zero = y_star_11(ell, tau, alpha = 0.0, lb_init = 0.0, ub_init = 2.0)
# print("y_star_zero:", y_star_zero)

y_star_vals = np.array([y_star_11(ell, tau, alpha, lb_init = 0.0, ub_init = 2.0) for alpha in alpha_vals])
y_star_vals_budget = alpha_vals
index_max = max([k for k in list(range(num_alpha_grid_points)) if alpha_vals[k] <= 1 - v_E/v_I])
# print("index_min:", index_min)
# print("alpha_vals[index_min]:", alpha_vals[index_min])
# print("y_star_vals_budget:", y_star_vals_budget)
# print()

# print("y_star_vals:", y_star_vals)
# print()

y_star_vals_discount = np.copy(y_star_vals)
for k in range(y_star_vals_discount.shape[0]):
    if k <= index_max:
        y_star_vals_discount[k] = 0.0

tau_thresh_for_alpha_3 = 2 * v_E * (ell(1) - ell(1 - 1E-6)) / 1E-6
index_for_alpha_3 = max([k for k in list(range(num_alpha_grid_points)) if y_star_vals[k] >= alpha_vals[k] ])
alpha_3 = (y_star_vals[index_for_alpha_3] + y_star_vals[index_for_alpha_3 + 1])/2

# print("y_star_vals_budget:", y_star_vals_budget)

# print("y_star_vals_budget:", y_star_vals_budget)


t_2 = time.time()
print("Time:", t_2 - t_1)

In [ ]:
print("1 - v^E / v^I:", 1 - v_E/v_I)
print("alpha_3:", alpha_3)

In [ ]:
plt.rcParams['text.usetex'] = True
# plt.rcParams['text.usetex'] = False

plt.plot(alpha_vals, y_star_vals_discount, 'red')
plt.plot(alpha_vals, y_star_vals_budget, 'blue')
# plt.scatter(alpha_vals, y_star_vals_discount, marker = 'o', s = 0.5, color = 'purple')
# plt.scatter(alpha_vals, y_star_vals_budget, marker = 'o', s = 0.5, color = 'black')
plt.xlim([0, 1.0])
plt.ylim([-0.2, 1.2])

plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel(r'$\displaystyle \alpha$', fontsize=24)
plt.ylabel(r'$\displaystyle y^C, y^D$', fontsize=24)
# plt.ylabel(r'$\displaystyle \ln(L^t - L^\star)$', fontsize=16)

# Scratch Work:

In [ ]:
# Test:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



# Frank-Wolfe Algorithm:

In [ ]:
# Below: For affine latency functions only:
def solve_CBCP_FW_11(tau, B, v_I, v_E, a = np.array([0.5, 1.0]), \
                       num_iters_max = 5000, error_bound = 1E-3, diffs_num_cols = 10, \
                       y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0]) ):
    
    # y = (\hat y_1 E, \tilde y_1 E, y_2 E, y_1 I, y_2 I)
    # a = (a_0, a_1)
    
#     print()    
#     print("Starting solve_CBCP_FW_11:")
#     print()
    
    # Initialize y_iters:
    y_iters = np.zeros((5, num_iters_max))
    
    y_el_var_current = y_init[0:3]
    y_in_var_current = y_init[3:]
    
    for k in range(num_iters_max):
        
        if k % 100 == 0:
            print("Inner iter:", k)
        
        # y_el: \hat y_1 E, \tilde y_1 E, y_2 E
        y_el_var = cp.Variable(3)
        # y_in: y_1 I, y_2 I
        y_in_var = cp.Variable(2)
        
        # Compute gradient:
        
        grad = np.zeros(5)
        grad[0] = a[1] * (y_el_var_current[0] + y_el_var_current[1] + y_in_var_current[0]) + a[0] + tau / v_E
        grad[1] = a[1] * (y_el_var_current[0] + y_el_var_current[1] + y_in_var_current[0]) + a[0]
        grad[2] = a[1] * (y_el_var_current[2] + y_in_var_current[1]) + a[0]
        grad[3] = a[1] * (y_el_var_current[0] + y_el_var_current[1] + y_in_var_current[0]) + a[0] + tau / v_I
        grad[4] = a[1] * (y_el_var_current[2] + y_in_var_current[1]) + a[0]
        
        ## Apply Frank-Wolfe:
        # Compute next iterate:
        
#         objective = cp.Minimize(0.5 * a[1] * (y_el_var[0] + y_el_var[1] + y_in_var[0])**2 \
#                                 + a[0] * (y_el_var[0] + y_el_var[1] + y_in_var[0]) \
#                                 + y_in_var[0] * tau / v_I + y_el_var[0] * tau / v_E \
#                                 + 0.5 * a[1] * (y_el_var[2] + y_in_var[1])**2 \
#                                 + a[0] * (y_el_var[2] + y_in_var[1]) )
        
        objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

        constraints = []
        constraints += [y_el_var >= 0, y_in_var >= 0]
        constraints += [cp.sum(y_el_var) == 1, cp.sum(y_in_var) == 1]
        constraints += [y_el_var[1] * tau <= B]
    
        prob = cp.Problem(objective, constraints)
        result = prob.solve()
        
#         print("grad:", grad)
#         print("y_el_var.value:", y_el_var.value)
#         print("y_in_var.value:", y_in_var.value)
#         print("y_el_var_current:", y_el_var_current)
#         print("y_in_var_current:", y_in_var_current)
        
        y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
        y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)
        
#         print("y_el_var_current (new):", y_el_var_current)
#         print("y_in_var_current (new):", y_in_var_current)
#         print()
        
        y_iters[0:3, k] = y_el_var_current
        y_iters[3:, k] = y_in_var_current
        
        if k >= diffs_num_cols + 2:
            diffs = np.linalg.norm(y_iters[:, k-diffs_num_cols-1:k-2] - y_iters[:, k-diffs_num_cols:k-1], axis = 0)

            if np.max(diffs) < error_bound:
                break
    
    return y_el_var_current, y_in_var_current
